# 🏁 Getting Started

1. Install [.net SDK 8.0](https://dotnet.microsoft.com/en-us/download/dotnet/8.0)
2. Install [Ollama](https://ollama.com/)
   - If you do not have permission to install software locally it can be requested from your manager as it is approved software
3. Ensure Ollama is up by running `ollama serve`. If you get an error then it is likely already running.

## 🦙 Let's test Ollama!

### Get the required packages from NuGet

In [ ]:
#r "nuget: Microsoft.Extensions.AI.Ollama, 9.0.1-preview.1.24570.5"
#r "nuget: Microsoft.SemanticKernel"
#r "nuget: Microsoft.SemanticKernel.Connectors.Ollama, 1.33.0-alpha"
#r "nuget: OllamaSharp"

### Get the correct URL for operating in a dev container or not

In [ ]:
var ollamaApiUrl = Environment.GetEnvironmentVariable("DOTNET_RUNNING_IN_CONTAINER") == "true"
  ? "http://host.docker.internal:11434"
  : "http://localhost:11434";

var languageModel = "llama3.2:3b";
var embeddingsModel = "nomic-embed-text";

### Get the models we need

In [ ]:
using OllamaSharp;
using (var ollama = new OllamaApiClient(ollamaApiUrl))
{
    await foreach (var status in ollama.PullModelAsync(languageModel))
        Console.WriteLine($"{status.Percent}% {status.Status}");

    await foreach (var status in ollama.PullModelAsync(embeddingsModel))
        Console.WriteLine($"{status.Percent}% {status.Status}");
}

### Test a question to the model

In [ ]:
using Microsoft.Extensions.AI;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

using (IChatClient client = new OllamaChatClient(new Uri(ollamaApiUrl), languageModel))
{
    await foreach (var update in client.CompleteStreamingAsync(
    [
        new ChatMessage(ChatRole.System, @"
            You are an expert on the works of Douglas Adams.
            Answer all questions using only quotations from his books and include citation.
            Do not include additional material outside of the answer and citation."),
        new ChatMessage(ChatRole.User, "What is the meaning of life the universe and everything?"),
    ]))
    {
        Console.Write(update);
    };
}